<a href="https://colab.research.google.com/github/AngeMervaneJdev/ML_AI_1/blob/main/tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==1.14

In [32]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [57]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.preprocessing import OneHotEncoder


In [60]:
from keras.datasets import mnist

In [38]:
print(tf. __version__)

2.8.0


#[Problème 1] Retour sur le scratch

Les étapes utiliser dans le scratch sont: 
- 1 - Initialisation du poids
- 2 - utiliser une booucle d'epoque pour 
- 3 - Multiplier le poids par l'entrée
- 4 - appliquer une fonction d'activation
- 5 - extimer et mettre à jour les poids

Mise e oeuvre avec tensorflow
- Initialisation des poids :
- Entrainement avec la boucle epoch

```
for epoch in range(1000):
    sess.run(train_step, feed_dict={
        x:x_train,
        t:y_train
    })
```
- Fonction d'activation
```
y = tf.sigmoid(tf.matmul(x, W) + b)
```
- Evaluation 
```
acc_val = sess.run(accuracy,
     feed_dict={
        x:x_train,
        t:y_train
    })
```

 

#[Problème 2] Considérez la correspondance entre le scratch et TensorFlow

In [53]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [54]:
"""
Classification binaire de l'ensemble de données Iris à l'aide d'un réseau de neurones implémenté dans TensorFlow

"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
tf.test.gpu_device_name() 
"""
tensorflowのバージョンを1.x系に変更した際は忘れずに
「!pip install tensorflow-gpu==1.14.0」でGPUのインストールをしておきましょう。
tf.test.gpu_device_name()でGPUの設定状態を確認し、認識されるかを確認します。
成功している場合はログが出力されます、認識されない場合は何も出力されません。
"""

# データセットの読み込み
df = pd.read_csv("Iris.csv")

# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# NumPy 配列に変換
X = np.array(X)
y = np.array(y)
# ラベルを数値に変換
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)



# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder(dtype="float", shape=[None, n_input])
Y = tf.placeholder(dtype="float", shape=[None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Epoch 0, loss : 14.4403, val_loss : 81.6895, acc : 0.625
Epoch 1, loss : 10.5877, val_loss : 57.9192, acc : 0.625
Epoch 2, loss : 6.7000, val_loss : 33.6513, acc : 0.625
Epoch 3, loss : 3.0198, val_loss : 10.7163, acc : 0.688
Epoch 4, loss : 0.6364, val_loss : 5.7109, acc : 0.562
Epoch 5, loss : 0.7567, val_loss : 11.6263, acc : 0.438
Epoch 6, loss : 0.5221, val_loss : 4.5032, acc : 0.688
Epoch 7, loss : 0.2289, val_loss : 2.7326, acc : 0.625
Epoch 8, loss : 0.2752, val_loss : 2.4822, acc : 0.688
Epoch 9, loss : 0.2005, val_loss : 2.7231, acc : 0.750
Epoch 10, loss : 0.1875, val_loss : 2.6312, acc : 0.750
Epoch 11, loss : 0.1578, val_loss : 2.2775, acc : 0.688
Epoch 12, loss : 0.1575, val_loss : 2.1782, acc : 0.688
Epoch 13, loss : 0.1519, val_loss : 2.1734, acc : 0.688
Epoch 14, loss : 0.1433, val_loss : 2.1291, acc : 0.688
Epoch 15, loss : 0.1350, val_loss : 2.0070, acc : 0.688
Epoch 16, loss : 0.1308, val_loss : 1.9295, acc : 0.688
Epoch 17, loss : 0.1272, val_loss : 1.8832, acc : 0

#[Problème 3] Créez un modèle d'Iris en utilisant les trois variables objectives.

In [58]:
df = pd.read_csv("Iris.csv")
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# NumPy 
X = np.array(X)
y = np.array(y)
# 
y[y == 'Iris-setosa'] = 0
y[y == "Iris-versicolor"] = 1
y[y == "Iris-virginica"] = 2

y = y.astype(np.int64)[:, np.newaxis]

#one-hotエンコーディング
enc = OneHotEncoder(categories='auto')
y = enc.fit_transform(y).A



# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 
X = tf.placeholder(dtype="float", shape=[None, n_input])
Y = tf.placeholder(dtype="float", shape=[None, n_classes])

# train
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):

    tf.random.set_random_seed(0)
    # 
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(tf.nn.softmax(logits), axis=1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: X_train, Y: y_train})
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss

        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Epoch 0, loss : 5.3173, val_loss : 31.7638, acc : 0.333
Epoch 1, loss : 1.6568, val_loss : 2.7251, acc : 0.542
Epoch 2, loss : 0.4273, val_loss : 1.6863, acc : 0.708
Epoch 3, loss : 0.1414, val_loss : 0.5362, acc : 0.875
Epoch 4, loss : 0.0957, val_loss : 0.4885, acc : 0.875
Epoch 5, loss : 0.0727, val_loss : 0.4696, acc : 0.875
Epoch 6, loss : 0.0570, val_loss : 0.4219, acc : 0.875
Epoch 7, loss : 0.0432, val_loss : 0.3696, acc : 0.875
Epoch 8, loss : 0.0308, val_loss : 0.3146, acc : 0.875
Epoch 9, loss : 0.0209, val_loss : 0.2740, acc : 0.875
Epoch 10, loss : 0.0150, val_loss : 0.2521, acc : 0.875
Epoch 11, loss : 0.0112, val_loss : 0.2408, acc : 0.875
Epoch 12, loss : 0.0085, val_loss : 0.2314, acc : 0.875
Epoch 13, loss : 0.0066, val_loss : 0.2230, acc : 0.875
Epoch 14, loss : 0.0054, val_loss : 0.2160, acc : 0.917
Epoch 15, loss : 0.0046, val_loss : 0.2130, acc : 0.917
Epoch 16, loss : 0.0041, val_loss : 0.2134, acc : 0.958
Epoch 17, loss : 0.0039, val_loss : 0.2121, acc : 0.958
E

#[Problème 4] Création d'un modèle de prix des logements

In [59]:
df = pd.read_csv("train.csv")
y = df['SalePrice']
X = df.loc[:, ['GrLivArea', 'YearBuilt']]

# NumPy 
X = np.array(X)
y = np.array(y)
# 
y = y.astype(np.int64)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.005
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 
X = tf.placeholder(dtype="float", shape=[None, n_input])
Y = tf.placeholder(dtype="float", shape=[None, n_classes])

# train
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):

    tf.random.set_random_seed(0)
    # 
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

                            
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.square(Y - logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Epoch 0, loss : 424048461.9101, val_loss : 2542237440.0000, acc : 1.000
Epoch 1, loss : 313543446.1670, val_loss : 2528067328.0000, acc : 1.000
Epoch 2, loss : 312923181.1906, val_loss : 2515195904.0000, acc : 1.000
Epoch 3, loss : 312321907.2891, val_loss : 2508679168.0000, acc : 1.000
Epoch 4, loss : 312325953.8844, val_loss : 2507447808.0000, acc : 1.000
Epoch 5, loss : 312201289.1135, val_loss : 2506053888.0000, acc : 1.000
Epoch 6, loss : 312164950.7495, val_loss : 2504360704.0000, acc : 1.000
Epoch 7, loss : 311859163.8887, val_loss : 2506366720.0000, acc : 1.000
Epoch 8, loss : 311742058.2441, val_loss : 2508734208.0000, acc : 1.000
Epoch 9, loss : 311600691.0835, val_loss : 2511005184.0000, acc : 1.000
Epoch 10, loss : 311752578.9122, val_loss : 2511408640.0000, acc : 1.000
Epoch 11, loss : 311739755.5118, val_loss : 2514722304.0000, acc : 1.000
Epoch 12, loss : 311712204.5396, val_loss : 2518148864.0000, acc : 1.000
Epoch 13, loss : 312159971.0493, val_loss : 2508695296.0000, 

#[Problème 5] Création d'un modèle MNIST

In [61]:
#データの読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#次元変換
y_train = y_train.astype(np.int)[:, np.newaxis]
y_test = y_test.astype(np.int)[:, np.newaxis]

#one-hotエンコーディング
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)

#データの変換
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

#分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

#ハイパーパラメータの設定
learning_rate = 0.01 
batch_size = 100 
num_epochs = 10 
n_hidden1 = 50 
n_hidden2 = 25 
n_input = X_train.shape[1] 
n_samples = X_train.shape[0] 
n_classes = 10 

#計算グラフに渡す引数の形を決める
X = tf.placeholder('float', [None, n_input])
Y = tf.placeholder('float', [None, n_classes])

#trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)


def example_net(x):
    '''
    単純な3層ニューラルネットワーク
    '''
    #重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    #全結合層
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] #addと同じ
    
    return layer_output

11501568/11490434 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Depr

In [62]:
#ネットワーク構造の読み込み
logits = example_net(X)

#目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

#最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(tf.nn.softmax(logits), axis=1))

#指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#variableの初期値
init = tf.global_variables_initializer()

In [63]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.0111, val_loss : 0.1889, acc : 0.773
Epoch 1, loss : 0.0013, val_loss : 0.1071, acc : 0.851
Epoch 2, loss : 0.0009, val_loss : 0.0813, acc : 0.885
Epoch 3, loss : 0.0007, val_loss : 0.0677, acc : 0.901
Epoch 4, loss : 0.0006, val_loss : 0.0587, acc : 0.916
Epoch 5, loss : 0.0005, val_loss : 0.0534, acc : 0.924
Epoch 6, loss : 0.0004, val_loss : 0.0482, acc : 0.931
Epoch 7, loss : 0.0004, val_loss : 0.0444, acc : 0.937
Epoch 8, loss : 0.0003, val_loss : 0.0418, acc : 0.939
Epoch 9, loss : 0.0003, val_loss : 0.0388, acc : 0.944
test_acc : 0.945
